In [38]:
import os
import pandas as pd
import numpy as np

In [39]:
face_features_path = r'D:\Body Pose Recognition\Dataset version 7 26-04-2022\train csv pyfeat'
body_features_path = r'D:\Body Pose Recognition\Dataset version 7 26-04-2022\train csv openpose'
multi_features_path = r'D:\Body Pose Recognition\Dataset version 7 26-04-2022\train csv multimodal'
num_features = 94

try:
    os.mkdir(multi_features_path)
except:
    print("Folder already exist")

In [40]:
def feature_reader(class_):
    data = []
    face_class_path = os.path.join(face_features_path, class_)
    body_class_path = os.path.join(body_features_path, class_)
    face_data = set(os.listdir(face_class_path))
    body_data = set(os.listdir(body_class_path))
    multi_data = face_data.intersection(body_data)
    face_remaining = face_data - multi_data
    body_remaining = body_data - multi_data
    
    # reading the data common in both face and body
    for file_name in multi_data:
        input_features = np.ones(num_features)*(1e-6)
        face_df = pd.read_csv(os.path.join(face_class_path, file_name), sep=',', header=None)
        body_df = pd.read_csv(os.path.join(body_class_path, file_name), sep=',', header=None)
        input_features[:71] = body_df.values[0, :]
        input_features[71:] = face_df.values[0, :]
        data.append(input_features)
    
    
    # reading the data present in only face
    for file_name in face_remaining:
        input_features = np.ones(num_features)*(1e-6)
        face_df = pd.read_csv(os.path.join(face_class_path, file_name), sep=',', header=None)
        input_features[71:] = face_df.values[0, :]
        data.append(input_features)
        
    # reading the data present in only body
    for file_name in body_remaining:
        input_features = np.ones(num_features)*(1e-6)
        body_df = pd.read_csv(os.path.join(body_class_path, file_name), sep=',', header=None)
        input_features[:71] = body_df.values[0, :]
        data.append(input_features)
    
    return data

base_folder = multi_features_path
output_classes = ['neg', 'pos']
class_index = {'pos' : 1, 'neg': 0}

for class_ in output_classes:
    behavior_index = class_index[class_]
    data = feature_reader(class_)
        
    df = pd.DataFrame(data)
    df['class'] = behavior_index
    df.to_csv(multi_features_path + '\\' + class_ + '.csv', sep=',',index=False, header=False)